In [ ]:
import easyocr
import cv2

### OVO JE PRESPORO ALI VEOMA KVALITETNO ~13s

reader = easyocr.Reader(['en'])
image = cv2.imread('slike/pic13.jpg')
results = reader.readtext(image, detail=0, paragraph=True)

print(results)

In [5]:
from qreader import QReader
import cv2

### QR DETECTION -> sporo? 2.8s

qreader = QReader()
image = cv2.cvtColor(cv2.imread("slike/pic13.jpg"), cv2.COLOR_BGR2RGB)
decoded_text = qreader.detect_and_decode(image=image)

str_text = ''.join(decoded_text)
instances =  [i for i, letter in enumerate(str_text) if letter == '\n']

print(str_text[instances[1]:instances[2]])
print(str_text[instances[2]:])


Datum:17.10.2023 17:45

Total:10,55


In [76]:
import cv2
import numpy as np
import pytesseract

### EDGE DETECTION & MORPHOLOGY

### trebam li dodati threshold? to provjeri
#binary_image = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 51, 7) #51, 15
# _, binary_image = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
### s obzirom da mi je prosli block bio threshold pa canny
### mislim da mi ne treba ali isprobaj pogledaj kako izgleda?

image = cv2.imread('slike/subject5.jpg', cv2.IMREAD_GRAYSCALE)

blurred = cv2.GaussianBlur(image, (5,5), 0)
edges = cv2.Canny(blurred, 50, 150)

kernel = np.ones((3,3), np.uint8)
kernel1 = np.ones((1,1), np.uint8)

closed_edges = cv2.morphologyEx(edges, cv2.MORPH_GRADIENT, kernel, iterations=2)
dilated_image = cv2.dilate(closed_edges, kernel1, iterations=1)

cv2.imwrite('edges.jpg', dilated_image)

### PRICE PROCESSING -> only right side (prices)

(imageHeight, imageWidth) = image.shape
imageROI = image[0:imageHeight, int(imageWidth/2):imageWidth]

result = pytesseract.image_to_string(imageROI, config='--psm 6 -c tessedit_char_whitelist=0123456789,E')

# print(pytesseract.image_to_string(dilated_image,
# config='--psm 4 --oem 3')) #testiraj oko config-a

In [77]:
import re
import pytesseract
import cv2

### proba da izvucem samo items

image = cv2.imread('edges.jpg', 0)
all_text = pytesseract.image_to_string(image, config=r'--psm 4')
matches = re.finditer(r'\d{12}', all_text)
for m in matches: z = m.end()
print(all_text[z:])


IBFH: AM028120
20. Pe » 14:58
Wrap Crispy Chicken 8, 00E
Medium Fries 1,50
Fanta Medius 1,50E
Ketchup Sachet 1,00E
VE: 17,00%
OSN. E: 10,26

E: 1,74
PDV: 1,74

dd7a86900f e522facf0a467ade7b0663

Sy,



In [45]:
import pandas as pd
import re

### DATA PROCESSING
pattern = r'\d+,\d{2}'
find_results = re.findall(pattern, result)

df = pd.DataFrame()
df = pd.concat([df, pd.Series()])
df = pd.concat([df, pd.Series(find_results)], axis=1)
df.columns = ["Items", "Price"]
print(df)
# df.to_csv("results.csv")

   Items Price
0    NaN  0,58
1    NaN  4,08
2    NaN  4,60
3    NaN  0,68


C:\Users\meho\AppData\Local\Temp\ipykernel_13116\43251792.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df, pd.Series()])
